# Modelamiento no supervisado en base a tópicos

In [1]:
import sys
sys.path.insert(0, '..')
from utils.preprocesamiento import StemmerTokenizer

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

In [2]:
from utils.cargar import df_caso
from utils.preprocesamiento import process_df

caso = 'laura'
df = df_caso(caso)

df = process_df(df,'comment','sel',verbose=True)

df = df.drop(columns=['user_id','team_id','gender','df','title','opt_left','opt_right','max_num','phase','time','curso'])

549 rows found with non string elements for column comment (11.03%)
Deleting 362 columns for which max target value is over 7 (7.27%)
4067 available rows after processing


In [3]:
from sklearn.model_selection import train_test_split

df_train, df_test, _, _ = train_test_split(df, df['sel'], test_size=.05, stratify=df['sel'], random_state=0)

In [4]:
tokenized_corpus = [tokenizador(document) for document in df_train['comment']]
tokenized_test = [tokenizador(document) for document in df_test['comment']]

In [5]:
import gensim
from gensim import corpora

# Create a dictionary from the tokenized corpus
dictionary = corpora.Dictionary(tokenized_corpus)

# Convert the tokenized corpus into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_corpus]

## Número óptimo de tópicos

In [6]:
from gensim.models import CoherenceModel

# Set the range of topic numbers to try
min_topics = 2
max_topics = 10
step_size = 1

# Initialize variables for best coherence score and best number of topics
best_coherence_score = -1
best_num_topics = -1

# Iterate over the range of topic numbers
for num_topics in range(min_topics, max_topics+1, step_size):
    # Train the LDA model
    lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=num_topics, id2word=dictionary, passes=10)
    
    # Calculate coherence score
    coherence_model = CoherenceModel(model=lda_model, texts=tokenized_corpus, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    
    # Check if coherence score is the best so far
    if coherence_score > best_coherence_score:
        best_coherence_score = coherence_score
        best_num_topics = num_topics

# Print the best number of topics
print(f"Best number of topics: {best_num_topics}")

Best number of topics: 4


In [7]:
%%time
lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=best_num_topics, id2word=dictionary, passes=10)

CPU times: user 11.2 s, sys: 0 ns, total: 11.2 s
Wall time: 11.2 s


In [8]:
# Print the generated topics
topics = lda_model.print_topics(num_topics=num_topics)
for topic in topics:
    print(topic)

(0, '0.051*"proyecto" + 0.026*"si" + 0.016*"renunciar" + 0.012*"impacto" + 0.011*"daño" + 0.010*"ambiente" + 0.010*"bien" + 0.009*"mejor" + 0.009*"Si" + 0.009*"correcto"')
(1, '0.014*"errores" + 0.014*"si" + 0.010*"proyecto" + 0.009*"importante" + 0.008*"trabajo" + 0.008*"Laura" + 0.007*"No" + 0.007*"aprender" + 0.006*"ser" + 0.006*"frustración"')
(2, '0.037*"trabajo" + 0.036*"tiempo" + 0.030*"familia" + 0.022*"vida" + 0.014*"debe" + 0.014*"importante" + 0.013*"si" + 0.013*"mas" + 0.012*"Laura" + 0.011*"bien"')
(3, '0.034*"error" + 0.017*"errores" + 0.015*"si" + 0.012*"haber" + 0.011*"renunciar" + 0.010*"aprender" + 0.010*"adecuado" + 0.009*"No" + 0.008*"Laura" + 0.008*"trabajo"')


In [9]:
print(' '.join(tokenized_test[0]) + '\n')

# Convert the tokenized document into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(tokenized_test[0])]

# Get the topic probabilities for the new document
topic_probs = lda_model.get_document_topics(doc_term_matrix)[0]

# Print the topic probabilities
for topic, prob in topic_probs:
    print(f"Topic {topic}: {prob}")

Personalmente encuentro importante vida personal trabajo dado pierden momentos realmente importan Sin embargo debe existir mínimo dedicación existir

Topic 0: 0.08290089666843414
Topic 1: 0.1859666109085083
Topic 2: 0.7174871563911438
Topic 3: 0.013645299710333347


## Visualización

In [10]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
pyLDAvis.display(vis_data)

## Correlaciones

In [11]:
from TM_utils import get_lda_embeddings
import pandas as pd

arr_test = get_lda_embeddings(lda_model,tokenizador,dictionary,df_test,'comment')
df_topics = pd.DataFrame(arr_test, columns=['Topic{}'.format(i+1) for i in range(len(topics))])

df_topics['sel'] = df_test['sel'].values

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (204,) + inhomogeneous part.

In [ ]:
from scipy.stats import pearsonr

# Only compute pearson prod-moment correlations between feature
# columns and target column
target_col_name = 'sel'
feature_target_corr = {}
for col in df_topics:
    if target_col_name != col:
        feature_target_corr[col + '_' + target_col_name] = \
            pearsonr(df_topics[col], df_topics[target_col_name])[0]
print("Feature-Target Correlations")
print(feature_target_corr)